# Prerequisites to run different code snippets below

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
import netCDF4 as nc
from netCDF4 import Dataset as NetCDFFile 
from datetime import datetime, timedelta
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans, SpectralClustering
from scipy import signal
import xarray as xr
import pylab
import operator
import pickle as pkl
from mpl_toolkits.basemap import Basemap

# generate a dictionary {station_id -> formatted_staion_name}
def get_station_dict():
    observation_data = nc.Dataset(observations)
    station_name_dict = {}
    for i in range(observation_data['station'].size):
        try:
            name = b''.join(list(observation_data['name'][i].compressed())).decode('utf-8')
        except AttributeError:
            name = b''.join(list(observation_data['name'][i])).decode('utf-8')
        station_name_dict[i] = name
    return station_name_dict

# plot the map of switzerland as background image
def plot_map(ax, rlat_min, rlat_max, rlon_min, rlon_max):
    dd = xr.open_dataset(data_path + "/c1ffsurf000_timeinvariant_lonlat.nc")
    dd.set_coords(["rlon", "rlat"], inplace=True)
    swiss_data = dd.sel(rlon=slice(rlon_min, rlon_max), rlat=slice(rlat_min, rlat_max))
    swiss_data.FR_LAND.isel(time = 0).plot.pcolormesh("rlon", "rlat", alpha=0.8, cmap=plt.cm.get_cmap('GnBu_r'), add_colorbar=False)
    swiss_data.HH.isel(time = 0).plot.pcolormesh("rlon", "rlat", alpha=0.6, cmap=plt.cm.get_cmap('YlGn'), add_colorbar=False)

# get time axis for plot
def get_time_axis(data_length, year_length):
    start=datetime.strptime('15100100', '%y%m%d%H')
    x_times = [start + timedelta(hours=t*3) for t in range(data_length)]
    return [x_times[i:i + year_length] for i in range(0, data_length, year_length)]

# filter out all corrupted measurements
def get_filtered_data(x,data):
    if len(x) != len(data):
        raise Exception('Timedate array and data do not have the same length!')
    return zip(*[(x_,d) for x_,d in zip(x,data) if d < 1e10])

# paths for data and plots
grid_size = 9
data_path = '/home/n1no/Documents/ethz/master_thesis/code/project/data'
experiment_data_path = data_path + '/preprocessed_data/grid_size_%s' % grid_size
error_data_path = experiment_data_path + '/baseline_station_error.nc'
time_invariant_data_per_station_path = experiment_data_path + '/time_invariant_data_per_station.pkl'
plot_output_path = '/home/n1no/Documents/ethz/master_thesis/code/project/results/nearest_grid_point_baseline/all_lead_times'
plot_output_path_no_lead_times = '/home/n1no/Documents/ethz/master_thesis/code/project/results/nearest_grid_point_baseline/no_lead_times'
observations = data_path + '/observations/meteoswiss_t2m_20151001-20171114.nc'

Coordinates:
  * station        (station) int64 2 3 4 6 7 8 9 10 11 13 14 15 17 18 19 20 ...
  * init_datetime  (init_datetime) float64 1.446e+18 1.449e+18 1.452e+18 ...
  * lead           (lead) int64 0 4 8 12 16 20 24 28 32

## Mean Error for each Station plotted per Lead Time

In [21]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean error per station with variance, if invalid errors are excluded (v.2.0)
#-------------------------------------------------------------------------------------------------------------------------
error_data_array = xr.open_dataarray(error_data_path)
num_stations, num_predictions, num_future_predictions = error_data_array.shape
error_data = error_data_array.data

for lead_time in range(num_future_predictions):
    print('Complete error of nearest grid point baseline for lead %s: %s K'
          % (lead_time, np.mean([abs(e) for e in error_data[:,:,lead_time].flatten() if np.abs(e) < 1e10])))
    N = num_stations
    station_error_means, station_error_std = np.zeros(num_stations), np.zeros(num_stations)
    for i in range(num_stations):
        valid_errors = [e for e in error_data[i,:,lead_time] if np.abs(e) < 1e10]
        station_error_means[i] = np.mean(valid_errors)
        station_error_std[i] = np.std(valid_errors)

    width = 0.35
    split = int(N/3)
    ind = np.arange(split)  # the x locations for the groups

    fig, axes = plt.subplots(nrows=3, ncols=1, sharey=True, figsize=(20, 10))
    for i in range(3):
        axes[i].bar(ind, station_error_means[ind+i*split], width,align='center', color='r', yerr=station_error_std[ind+i*split])
        axes[i].set_xlim(-width,split)
        axes[i].set_xticks(ind)
        axes[i].set_xticklabels(['S' + str(idx+i*split) for idx in ind])
        axes[i].grid('On')

    # add some text for labels, title and axes ticks
    plt.ylim((-5,7))
    plt.yticks(range(-5,7))
    axes[1].set_ylabel('Error [K]')
    axes[0].set_title('Mean Temperature Error for each Station over compelte Data for lead %s' % (lead_time if lead_time == 0 else "+%s" % lead_time))
    fig.savefig(plot_output_path + '/nearest_grid_prediction_error_per_station_lead_%s.png' % lead_time)
    plt.close()

Complete error of nearest grid point baseline for lead 0: 1.524599704929745 K


/home/n1no/anaconda3/envs/python3_thesis/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


Complete error of nearest grid point baseline for lead 1: 1.5687164835043077 K
Complete error of nearest grid point baseline for lead 2: 1.6734138525301425 K
Complete error of nearest grid point baseline for lead 3: 1.7951230954805293 K
Complete error of nearest grid point baseline for lead 4: 1.8338030059496346 K
Complete error of nearest grid point baseline for lead 5: 1.8159432719331556 K
Complete error of nearest grid point baseline for lead 6: 1.8469758590503835 K
Complete error of nearest grid point baseline for lead 7: 1.9230893156200652 K
Complete error of nearest grid point baseline for lead 8: 1.9018908942599277 K


In [23]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean absolute error per station with variance, if invalid errors are excluded  (v.2.0)
#-------------------------------------------------------------------------------------------------------------------------
error_data_array = xr.open_dataarray(error_data_path)
num_stations, num_predictions, num_future_predictions = error_data_array.shape
error_data = error_data_array.data

for lead_time in range(num_future_predictions):
    print('Complete error of nearest grid point baseline for lead %s: %s K'
          % (lead_time, np.mean([abs(e) for e in error_data[:,:,lead_time].flatten() if np.abs(e) < 1e10])))
    N = num_stations
    station_error_means, station_error_std = np.zeros(num_stations), np.zeros(num_stations)
    for i in range(num_stations):
        valid_errors = [abs(e) for e in error_data[i,:,lead_time] if np.abs(e) < 1e10]
        station_error_means[i] = np.mean(valid_errors)
        station_error_std[i] = np.std(valid_errors)

    width = 0.35
    split = int(N/3)
    ind = np.arange(split)  # the x locations for the groups

    fig, axes = plt.subplots(nrows=3, ncols=1, sharey=True, figsize=(20, 10))
    for i in range(3):
        axes[i].bar(ind, station_error_means[ind+i*split], width,align='center', color='r', yerr=station_error_std[ind+i*split])
        axes[i].set_xlim(-width,split)
        axes[i].set_xticks(ind)
        axes[i].set_xticklabels(['S' + str(idx+i*split) for idx in ind])
        axes[i].grid('On')

    # add some text for labels, title and axes ticks
    plt.ylim((0,7))
    plt.yticks(range(7))
    axes[1].set_ylabel('Error [K]')
    axes[0].set_title('Mean absolute Temperature Error for each Station over compelte Data for lead %s' % (lead_time if lead_time == 0 else "+%s" % lead_time))
    fig.savefig(plot_output_path + '/nearest_grid_absolute_prediction_error_per_station_lead_%s.png' % lead_time)
    plt.close()

Complete error of nearest grid point baseline for lead 0: 1.524599704929745 K


/home/n1no/anaconda3/envs/python3_thesis/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


Complete error of nearest grid point baseline for lead 1: 1.5687164835043077 K
Complete error of nearest grid point baseline for lead 2: 1.6734138525301425 K
Complete error of nearest grid point baseline for lead 3: 1.7951230954805293 K
Complete error of nearest grid point baseline for lead 4: 1.8338030059496346 K
Complete error of nearest grid point baseline for lead 5: 1.8159432719331556 K
Complete error of nearest grid point baseline for lead 6: 1.8469758590503835 K
Complete error of nearest grid point baseline for lead 7: 1.9230893156200652 K
Complete error of nearest grid point baseline for lead 8: 1.9018908942599277 K


## Mean Error over Lead Time per Initialization Time plotted per Station

In [26]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean error by initialization time and lead time (v.2.0)
#-------------------------------------------------------------------------------------------------------------------------

station_name_dict = get_station_dict()

error_data_array = xr.open_dataarray(error_data_path)
num_stations, num_predictions, num_future_predictions = error_data_array.shape
error_data = error_data_array.data

x = range(0,num_future_predictions)
initialization_times = range(0,24,3)

for station in range(num_stations):
    
    # plot error per station by lead
    fig, axes = plt.subplots(len(initialization_times), sharex=True, sharey=True, figsize=(20, 20))
    max_difference_per_station = -np.inf
    for idx, init_time in enumerate(initialization_times):
        error_per_init_time = []
        for lead in range(num_future_predictions):
            error_per_init_time += [np.mean([e for e in error_data[station,idx::8,lead].flatten() if np.abs(e) < 1e10])]
        axes[idx].plot(x, error_per_init_time)
        axes[idx].set_ylabel('Error [°C]')
        axes[idx].annotate('Init. Time: %s' % init_time,  xy=(0.005, 0.85), xycoords="axes fraction", fontsize=16)
        
        max_difference_per_init_time = (max(error_per_init_time) - min(error_per_init_time))
        axes[idx].annotate('Max Temp. Diff.: %.2f °C' % max_difference_per_init_time,
                   xy=(1, 0.95), horizontalalignment='right', verticalalignment='top', color='black',
                 xycoords="axes fraction", fontsize=16)
        
        max_difference_per_station = np.maximum(max_difference_per_station, max_difference_per_init_time)
    plt.xticks(x, ['0h'] + ['+%sh' % i for i in x[1:]])
    plt.xlabel('Lead Time')
    plt.suptitle('%s - Error by Initialization & Lead Time' % station_name_dict[station], fontsize=22)
    axes[0].annotate('Total Max Temp. Diff.: %.2f °C' % max_difference_per_station,
                       xy=(1, 1.05), horizontalalignment='right', verticalalignment='bottom', color='red',
                     xycoords="axes fraction", fontsize=16)

    fig.tight_layout(rect=[0, 0.03, 1, 0.97])
    fig.savefig(plot_output_path + '/error_by_init_time_and_lead_time/Station_%s.png' % station)
    plt.close()

In [29]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean absolute error by initialization time and lead time (v.2.0)
#-------------------------------------------------------------------------------------------------------------------------

station_name_dict = get_station_dict()

error_data_array = xr.open_dataarray(error_data_path)
num_stations, num_predictions, num_future_predictions = error_data_array.shape
error_data = error_data_array.data

x = range(0,num_future_predictions)
initialization_times = range(0,24,3)

for station in range(num_stations):
    
    # plot error per station by lead
    fig, axes = plt.subplots(len(initialization_times), sharex=True, sharey=True, figsize=(20, 20))
    error_per_station = []
    for idx, init_time in enumerate(initialization_times):
        error_per_init_time = []
        for lead in range(num_future_predictions):
            error_per_init_time += [np.mean([abs(e) for e in error_data[station,idx::8,lead].flatten() if np.abs(e) < 1e10])]
        axes[idx].plot(x, error_per_init_time)
        axes[idx].set_ylabel('Error [°C]')
        axes[idx].annotate('Init. Time: %s' % init_time,  xy=(0.005, 0.85), xycoords="axes fraction", fontsize=16)
        error_per_station += error_per_init_time
    plt.xticks(x, ['0h'] + ['+%sh' % i for i in x[1:]])
    plt.xlabel('Lead Time')
    plt.suptitle('%s - Absolute Error by Initialization & Lead Time' % station_name_dict[station], fontsize=22)
    axes[0].annotate('Total Max Temp. Diff.: %.2f °C' % max_difference_per_station,
                       xy=(1, 1.05), horizontalalignment='right', verticalalignment='bottom', color='red',
                     xycoords="axes fraction", fontsize=16)

    fig.tight_layout(rect=[0, 0.03, 1, 0.97])
    fig.savefig(plot_output_path + '/absolute_error_by_init_time_and_lead_time/Station_%s.png' % station)
    plt.close()

## Mean Error over Lead Times (averaged over all Init. Times) plotted per Station

In [30]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean error by lead time (v.2.0)
#-------------------------------------------------------------------------------------------------------------------------

station_name_dict = get_station_dict()

error_data_array = xr.open_dataarray(error_data_path)
num_stations, num_predictions, num_future_predictions = error_data_array.shape
error_data = error_data_array.data

x = range(0,num_future_predictions)
initialization_times = range(0,24,3)

# calculate the error per station and lead, and the overall min and max to scale the plots
min_error, max_error = np.inf, -np.inf
error = np.zeros((num_stations, num_future_predictions))
for station in range(num_stations):
    for lead in range(num_future_predictions):
        error[station, lead] = np.mean([e for e in error_data[station,:,lead].flatten() if np.abs(e) < 1e10])
        min_error = np.minimum(min_error, error[station, lead])
        max_error = np.maximum(max_error, error[station, lead])

# plot the data
for station in range(num_stations):
    fig = plt.figure(figsize=(20,10))
    plt.plot(x, error[station])
    max_diff = np.max(error[station]) - np.min(error[station])
    plt.text(30, max_error, 'Max Temp. Diff.: %.2f °C' % max_diff,
        verticalalignment='top', horizontalalignment='right',
        color='r', fontsize=22)
    plt.xticks(x, ['0h'] + ['+%sh' % i for i in x[1:]])
    plt.xlabel('Lead Time')
    # plt.ylim((min_error,max_error))
    plt.ylabel('Error [°C]')
    plt.suptitle('%s - Error by Lead Time' % station_name_dict[station], fontsize=22)
    fig.tight_layout(rect=[0, 0.03, 1, 0.97])
    fig.savefig(plot_output_path + '/error_by_lead_time/Station_%s.png' % station)
    plt.close()

In [31]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean absolute error by lead time
#-------------------------------------------------------------------------------------------------------------------------

station_name_dict = get_station_dict()

error_data_array = xr.open_dataarray(error_data_path)
num_stations, num_predictions, num_future_predictions = error_data_array.shape
error_data = error_data_array.data

x = range(0,num_future_predictions)
initialization_times = range(0,24,3)

# calculate the error per station and lead, and the overall min and max to scale the plots
min_error, max_error = np.inf, -np.inf
error = np.zeros((num_stations, num_future_predictions))
for station in range(num_stations):
    for lead in range(num_future_predictions):
        error[station, lead] = np.mean([abs(e) for e in error_data[station,:,lead].flatten() if np.abs(e) < 1e10])
        min_error = np.minimum(min_error, error[station, lead])
        max_error = np.maximum(max_error, error[station, lead])

# plot the data
for station in range(num_stations):
    fig = plt.figure(figsize=(20,10))
    plt.plot(x, error[station])
    max_diff = np.max(error[station]) - np.min(error[station])
    plt.text(30, max_error, 'Max Temp. Diff.: %.2f °C' % max_diff,
        verticalalignment='top', horizontalalignment='right',
        color='r', fontsize=22)
    plt.xticks(x, ['0h'] + ['+%sh' % i for i in x[1:]])
    plt.xlabel('Lead Time')
    plt.ylim((min_error,max_error))
    plt.ylabel('Error [°C]')
    plt.suptitle('%s - Absolute Error by Lead Time' % station_name_dict[station], fontsize=22)
    fig.tight_layout(rect=[0, 0.03, 1, 0.97])
    fig.savefig(plot_output_path + '/absolute_error_by_lead_time/Station_%s.png' % station)
    plt.close()

## Mean Error over Initialization Times (averaged over all Lead Times) plotted per Station

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean error by initialization time
#-------------------------------------------------------------------------------------------------------------------------

station_name_dict = get_station_dict()

error_data = np.load(error_data_path)
num_stations, num_future_predictions, num_predictions = error_data.shape

initialization_times = range(0,24,3)
num_initialization_times = len(initialization_times)

# calculate the error per station and initialization time, and the overall min and max to scale the plots
min_error, max_error = np.inf, -np.inf
error = np.zeros((num_stations, num_initialization_times))
for station in range(num_stations):
    for idx, init_time in enumerate(initialization_times):
        error[station, idx] += np.mean([e for e in error_data[station,:24,idx::8].flatten() if np.abs(e) < 1e10])
        min_error = np.minimum(min_error, error[station, idx])
        max_error = np.maximum(max_error, error[station, idx])

# plot error per station by initialization time
for station in range(num_stations):
    fig = plt.figure(figsize=(20, 10))
    plt.plot(initialization_times, error[station])
    max_diff = np.max(error[station]) - np.min(error[station])
    plt.text(20, np.max(error[station]), 'Max Temp. Diff.: %.2f °C' % max_diff,
        verticalalignment='top', horizontalalignment='right',
        color='r', fontsize=22)
    #plt.ylim((min_error,max_error))
    plt.ylabel('Error [°C]')
    plt.xlabel('Initialization Time')
    plt.suptitle('%s - Error by Initialization Time for next 24h' % station_name_dict[station], fontsize=22)
    fig.tight_layout(rect=[0, 0.03, 1, 0.97])
    fig.savefig(plot_output_path + '/error_by_init_time/Station_%s.png' % station)
    plt.close()

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean absolute error by initialization time
#-------------------------------------------------------------------------------------------------------------------------

station_name_dict = get_station_dict()

error_data = np.load(error_data_path)
num_stations, num_future_predictions, num_predictions = error_data.shape

initialization_times = range(0,24,3)

# calculate the error per station and initialization time, and the overall min and max to scale the plots
min_error, max_error = np.inf, -np.inf
error = np.zeros((num_stations, num_initialization_times))
for station in range(num_stations):
    for idx, init_time in enumerate(initialization_times):
        error[station, idx] += np.mean([abs(e) for e in error_data[station,:24,idx::8].flatten() if np.abs(e) < 1e10])
        min_error = np.minimum(min_error, error[station, idx])
        max_error = np.maximum(max_error, error[station, idx])

# plot error per station by initialization time
for station in range(num_stations):
    fig = plt.figure(figsize=(20, 10))
    plt.plot(initialization_times, error[station])
    max_diff = np.max(error[station]) - np.min(error[station])
    plt.text(20, np.max(error[station]), 'Max Temp. Diff.: %.2f °C' % max_diff,
        verticalalignment='top', horizontalalignment='right',
        color='r', fontsize=22)
    #plt.ylim((min_error,max_error))
    plt.ylabel('Error [°C]')
    plt.xlabel('Initialization Time')
    plt.suptitle('%s - Absolute Error by Initialization Time for next 24h' % station_name_dict[station], fontsize=22)
    fig.tight_layout(rect=[0, 0.03, 1, 0.97])
    fig.savefig(plot_output_path + '/absolute_error_by_init_time/Station_%s.png' % station)
    plt.close()

## Mean Error over Lead Times (averaged over all Init. Times & Stations)

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean error over lead times for all stations combined
#-------------------------------------------------------------------------------------------------------------------------

error_data = np.load(error_data_path)
num_stations, num_future_predictions, num_predictions = error_data.shape

error_data = np.swapaxes(error_data, 1, 0)

future_predictions = range(num_future_predictions)
mean_error_per_lead = np.zeros(num_future_predictions)
for lead in future_predictions:
    mean_error_per_lead[lead] = np.mean([e for e in error_data[lead].flatten() if np.abs(e) < 1e10])

regr = linear_model.LinearRegression()
regr.fit(np.array(future_predictions).reshape(-1,1), np.array(mean_error_per_lead))

# plot data and trend line
fig = plt.figure(figsize=(20,10))
real_data = plt.plot(future_predictions, mean_error_per_lead, 'b')
reg_data = plt.plot(future_predictions, regr.predict(np.array(future_predictions).reshape(-1,1)), 'r--', alpha=0.5)

# add text label of absolute error difference between min and max
max_diff = np.max(mean_error_per_lead) - np.min(mean_error_per_lead)
plt.text(3, np.max(mean_error_per_lead), 'Max Temp. Diff.: %.2f °C' % max_diff,
    verticalalignment='top', horizontalalignment='left',
    color='r', fontsize=22)

# plot cosmetics
plt.xticks(future_predictions, ['0h'] + ['+%sh' % i for i in future_predictions[1:]])
plt.xlabel('Lead Time')
plt.ylabel('Error [K]')
plt.title('Mean Temperature Error over all Stations for Lead Times')
plt.grid('On')
fig.savefig(plot_output_path + '/nearest_grid_prediction_error_all_stations_over_lead_times.png')
plt.close()

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot mean absolute error over lead times for all stations combined
#-------------------------------------------------------------------------------------------------------------------------

error_data = np.load(error_data_path)
num_stations, num_future_predictions, num_predictions = error_data.shape

error_data = np.swapaxes(error_data, 1, 0)

future_predictions = range(num_future_predictions)
mean_error_per_lead = np.zeros(num_future_predictions)
for lead in future_predictions:
    mean_error_per_lead[lead] = np.mean([abs(e) for e in error_data[lead].flatten() if np.abs(e) < 1e10])

regr = linear_model.LinearRegression()
regr.fit(np.array(future_predictions).reshape(-1,1), np.array(mean_error_per_lead))

# plot data and trend line
fig = plt.figure(figsize=(20,10))
real_data = plt.plot(future_predictions, mean_error_per_lead, 'b')
reg_data = plt.plot(future_predictions, regr.predict(np.array(future_predictions).reshape(-1,1)), 'r--', alpha=0.5)

# add text label of absolute error difference between min and max
max_diff = np.max(mean_error_per_lead) - np.min(mean_error_per_lead)
plt.text(3, np.max(mean_error_per_lead), 'Max Temp. Diff.: %.2f °C' % max_diff,
    verticalalignment='top', horizontalalignment='left',
    color='r', fontsize=22)

# plot cosmetics
plt.xticks(future_predictions, ['0h'] + ['+%sh' % i for i in future_predictions[1:]])
plt.xlabel('Lead Time')
plt.ylabel('Error [K]')
plt.title('Mean absolute Temperature Error over all Stations for Lead Times')
plt.grid('On')
fig.savefig(plot_output_path + '/nearest_grid_absolute_prediction_error_all_stations_over_lead_times.png')
plt.close()

## Periodogram of Error over complete Measurement Period plottet for each Station

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot periodogram of error per station
#-------------------------------------------------------------------------------------------------------------------------

# stations with heavy diurnal variantions
diurnal_stations = []

error_data = np.load(error_data_path)
n_stations, _, data_length = error_data.shape
sample_rate = 8
station_name_dict = get_station_dict()
for i in range(144):
    fig, axes = plt.subplots(1, figsize=(20, 10))
    f, Pxx_den = signal.periodogram(error_data[i,0,:], sample_rate)
    relevant_frequencies = [idx for idx, y in enumerate(Pxx_den) if y > 500]
    if len(relevant_frequencies) > 0 and len(relevant_frequencies) < 10:
        diurnal_stations += [(i, station_name_dict[i])]
        print('Station %s\t%s' % (i, station_name_dict[i]))
    axes.plot(f, Pxx_den)
    axes.set_xlabel('Frequency [1/day]')
    plt.suptitle(station_name_dict[i] + ' (Station %s)' % str(i+1))
    fig.savefig(plot_output_path + '/periodogram/error_periodogram_station_%s' % str(i))
    plt.close()

## Mean Error per Station plottet on geographic Map, Location of corrupted Stations, Error vs. Height Scatter Plot (with xarray and TOPOdata)

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot station by error on map
#-------------------------------------------------------------------------------------------------------------------------

# load time invariant data per station, e.g. station attributes itself, and attributes of surrounding grid points
f = open(time_invariant_data_per_station_path, "rb")
time_invariant_data_per_station = pkl.load(f)
f.close()
# time invariant observation data
OBS_inv = nc.Dataset(data_path + "/c1ffsurf000_timeinvariant_lonlat.nc")
# load baseline error data
error_data = np.load(error_data_path).squeeze()
# dictionary with index of station in [0,143] to index of station in filtered data array [0,<=143]
idx_station_dict = {}
# filter out stations with corrupted measurements
filtered_error_data = None
stations_with_errors = []
for i in range(error_data.shape[0]):
    if np.max(error_data[i]) > 1e10:
        stations_with_errors += [i]
        continue
    if filtered_error_data is None:
        filtered_error_data = error_data[i]
        idx_station_dict[0] = i
    else:
        filtered_error_data = np.vstack((filtered_error_data, error_data[i]))
        idx_station_dict[len(filtered_error_data)-1] = i

# time invarioant (x,y,z)-coordiantes of stations
station_x = [OBS_inv['rlon'][time_invariant_data_per_station[idx_station_dict[i]]['self']['closest_grid_point'][1]] for i in range(len(filtered_error_data))]
station_y = [OBS_inv['rlat'][time_invariant_data_per_station[idx_station_dict[i]]['self']['closest_grid_point'][0]] for i in range(len(filtered_error_data))]
station_z = [time_invariant_data_per_station[idx_station_dict[i]]['self']['height'] for i in range(len(filtered_error_data))]

error_station_x = [OBS_inv['rlon'][time_invariant_data_per_station[stations_with_errors[i]]['self']['closest_grid_point'][1]] for i in range(len(stations_with_errors))]
error_station_y = [OBS_inv['rlat'][time_invariant_data_per_station[stations_with_errors[i]]['self']['closest_grid_point'][0]] for i in range(len(stations_with_errors))]

rlat_min, rlat_max, rlon_min, rlon_max = -1.3, 1, -3, 0.5

# plot the absolute error of each station in a heat map
fig = plt.figure(figsize=(26,15))
ax = fig.add_subplot(111)
plot_map(ax, rlat_min, rlat_max, rlon_min, rlon_max)
cax = ax.scatter(station_x, station_y, c=np.mean(np.abs(filtered_error_data),1), s=200,
                 cmap='YlOrBr', norm=mpl.colors.Normalize(vmin=0, vmax=3))
cbar = plt.colorbar(cax, format=ticker.FuncFormatter(lambda x, pos: "%.1f °C" % x))
ax.set_title('Error of stations', fontsize=22)
ax.grid('off')
plt.axis('scaled')
plt.xlabel('')
plt.ylabel('')
fig.savefig(plot_output_path + '/absolute_station_error_on_map.png')
plt.close()

# plot the error of each station in a heat map
fig = plt.figure(figsize=(26,15))
ax = fig.add_subplot(111)
plot_map(ax, rlat_min, rlat_max, rlon_min, rlon_max)
cax = ax.scatter(station_x, station_y, c=np.mean(filtered_error_data,1), s=200,
                 cmap='bwr', norm=mpl.colors.Normalize(vmin=-3, vmax=3))
cbar = plt.colorbar(cax, format=ticker.FuncFormatter(lambda x, pos: "%.1f °C" % x))
ax.set_title('Error of stations', fontsize=22)
ax.grid('off')
plt.axis('scaled')
plt.xlabel('')
plt.ylabel('')
fig.savefig(plot_output_path + '/station_error_on_map.png')
plt.close()

# Stations with measurment error
fig = plt.figure(figsize=(26,15))
ax = fig.add_subplot(111)
plot_map(ax, rlat_min, rlat_max, rlon_min, rlon_max)
ax.scatter(error_station_x, error_station_y, s=200, color='r')
ax.set_title('Stations with Errors in Measurement Sequence', fontsize=22)
ax.grid('off')
plt.axis('scaled')
plt.xlabel('')
plt.ylabel('')
fig.savefig(plot_output_path + '/corrupted_stations.png')
plt.close()
            
# Station absolute Error vs. Station Height Scatter plot
fig = plt.figure(figsize=(20,10))
x,y = list(zip(*sorted(zip(np.mean(np.abs(filtered_error_data),1), station_z))))
plt.scatter(x,y)


# calc the trendline
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
# plot trend line
plt.plot(x,p(x),"r--")

plt.xlabel('Error [°C]')
plt.ylabel('Height (normalized)')
plt.suptitle('Absolute Error vs. Station Height Plot')
fig.savefig(plot_output_path + '/absolute_station_error_agains_height.png')
plt.close()

# Station Error vs. Station Height Scatter plot
fig = plt.figure(figsize=(20,10))
x,y = list(zip(*sorted(zip(np.mean(filtered_error_data,1), station_z))))
plt.scatter(x,y)

# calc the trendline
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
# plot trend line
plt.plot(x,p(x),"r--")

plt.xlabel('Error [°C]')
plt.ylabel('Height (normalized)')
plt.suptitle('Error vs. Station Height Plot')
fig.savefig(plot_output_path + '/station_error_agains_height.png')
plt.close()

## Background Map Plots without error data with "BaseMap"

In [ ]:
nc_file = NetCDFFile(data_path + "/c1ffsurf000_timeinvariant_lonlat.nc")

lat = nc_file.variables['lat'][:]
lon = nc_file.variables['lon'][:]
HH = nc_file.variables['HH'][:]
FR_LAND = nc_file.variables['FR_LAND'][:]
SOILTYP = nc_file.variables['SOILTYP'][:]

fig = plt.figure(figsize=(20,10))
map=Basemap(projection="lcc",resolution="f",width=4E5,height=2.5E5,
                             lon_0=8.17,lat_0=46.8,fix_aspect=False)
map.drawcountries(zorder=1,color="black", linewidth=1)
map.shadedrelief(scale=2.5)
map.drawcoastlines(color="black",linewidth=1.2)
map.drawrivers(linewidth=0.5,color="blue")

plt.show()
plt.close()

In [ ]:
nc = NetCDFFile(data_path + "/c1ffsurf000_timeinvariant_lonlat.nc")

lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
HH = nc.variables['HH'][:]
FR_LAND = nc.variables['FR_LAND'][:]
SOILTYP = nc.variables['SOILTYP'][:]

fig = plt.figure(figsize=(20,10))
m = Basemap(llcrnrlon=5.8,llcrnrlat=45.8,urcrnrlon=11.,urcrnrlat=47.9, epsg=5520)
m.arcgisimage(service='ESRI_Imagery_World_2D', xpixels = 3000, verbose= True)
m.drawcountries(zorder=1,color="black", linewidth=1)
plt.show()

## Mean Error per Hour plottet (averaged over all Stations)

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Error plot per hour
#-------------------------------------------------------------------------------------------------------------------------

hour_error_dict = {
    0 : [],
    3 : [],
    6 : [],
    9 : [],
    12 : [],
    15 : [],
    18 : [],
    21 : []
}

error_data = np.load(error_data_path).squeeze()
error_mask = np.where(error_data < 1e10)

hours = np.array([[(i*3) % 24 for i in range(error_data.shape[1])] for j in range(error_data.shape[0])])
filtered_error_hour = np.dstack((error_data, hours))[error_mask]

for idx, d in enumerate(filtered_error_hour):
    hour_error_dict[d[1]] += [d[0]]
    
num_hours = len(hour_error_dict.keys())
mean = np.zeros(num_hours)
std = np.zeros(num_hours)
for idx, key in enumerate(hour_error_dict.keys()):
    mean[idx] = np.mean(hour_error_dict[key])
    std[idx] = np.std(hour_error_dict[key])

fig = plt.figure(figsize=(20,10))
plt.errorbar(hour_error_dict.keys(), mean, std)
plt.xlabel('Hour [h]')
plt.ylabel('Error [°C]')
plt.suptitle('Error agains Hour')
fig.savefig(plot_output_path + '/all_stations_error_against_hour.png')
plt.close()

mean = np.zeros(num_hours)
std = np.zeros(num_hours)
for idx, key in enumerate(hour_error_dict.keys()):
    mean[idx] = np.mean(np.abs(hour_error_dict[key]))
    std[idx] = np.std(np.abs(hour_error_dict[key]))

fig = plt.figure(figsize=(20,10))
plt.errorbar(hour_error_dict.keys(), mean, std)
plt.xlabel('Hour [h]')
plt.ylabel('Error [°C]')
plt.suptitle('Absolute Error agains Hour')
fig.savefig(plot_output_path + '/all_stations_absolute_error_against_hour.png')
plt.close()

## Mean Error per Hour plotted for each Station

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Error plot per hour per station
#-------------------------------------------------------------------------------------------------------------------------

error_data = np.load(error_data_path).squeeze()[:,1,:]
hours = np.array([(i*3) % 24 for i in range(error_data.shape[1])])

station_name_dict = get_station_dict()

# mean error for each station per hour
station_mean_error = np.zeros((144,8))

for station in range(144):
    hour_error_dict = {
        0 : [],
        3 : [],
        6 : [],
        9 : [],
        12 : [],
        15 : [],
        18 : [],
        21 : []
    }
    
    error_mask = np.where(error_data[station,:] < 1e10)
    filtered_error_hour = np.dstack((error_data[station,:], hours)).squeeze()[error_mask]

    for idx, d in enumerate(filtered_error_hour):
        hour_error_dict[d[1]] += [d[0]]

    num_hours = len(hour_error_dict.keys())
    mean = np.zeros(num_hours)
    std = np.zeros(num_hours)
    for idx, key in enumerate(hour_error_dict.keys()):
        mean[idx] = np.mean(hour_error_dict[key])
        std[idx] = np.std(hour_error_dict[key])
    
    # could be reused for bias corrigated base line error
    station_mean_error[station] = mean

max_error, min_error = np.max(station_mean_error), np.min(station_mean_error)

for station in range(144):
    fig = plt.figure(figsize=(14,8))
    plt.plot(hour_error_dict.keys(), station_mean_error[station])
    
    max_difference = np.max(station_mean_error[station]) - np.min(station_mean_error[station])
    plt.annotate('Max Temp. Diff.: %.2f °C' % max_difference,
           xy=(1, 1.01), horizontalalignment='right', verticalalignment='bottom', color='red',
         xycoords="axes fraction", fontsize=16)
    
    plt.ylim((min_error, max_error))
    plt.xlabel('Hour [h]')
    plt.ylabel('Error [°C]')
    plt.title('%s - Error agains Hour' % station_name_dict[station], fontsize=22)
    plt.grid('On')
    fig.savefig(plot_output_path_no_lead_times + '/error_per_hour/error_against_hour_station_%s.png' % str(station))
    plt.close()

## Error over complete Measurement Period plotted by Station

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot error per station over years, if invalid errors are excluded
#-------------------------------------------------------------------------------------------------------------------------

error_data = np.load(error_data_path)
n_stations, _, data_length = error_data.shape
year_length=365*8
x_times = get_time_axis(data_length,year_length)
station_name_dict = get_station_dict()
# loop over stations
for station in range(n_stations):
    data = [error_data[station,0,i:i + year_length] for i in range(0, data_length, year_length)]
    fig, axes = plt.subplots(len(data), figsize=(20, 20))
    min_y = np.min([e for e in error_data[station,0,:].flatten() if e < 1e10])
    max_y = np.max([e for e in error_data[station,0,:].flatten() if e < 1e10])
    for i in range(len(data)):
        # get start/end of the yearly periods
        start = x_times[i][0]
        end = x_times[i][0] + timedelta(days=365)
        # get the data and corresponding datetimes without the faulty samples
        x,error_filtered_data = get_filtered_data(x_times[i], data[i])
        # plot
        axes[i].plot(x, error_filtered_data, color='b', alpha=0.2)
        try:
        # get filtered line to show trend
            b, a = signal.butter(3, 0.015)
            y = signal.filtfilt(b, a, error_filtered_data)
            axes[i].plot(x, y ,color='r')
        except ValueError:
            continue
        # set x-/y-limits
        axes[i].set_xlim((start,end))
        axes[i].set_ylim((min_y,max_y))

    axes[len(data)-1].set_xlabel('Days')
    plt.suptitle('%s (Station %s)' % (station_name_dict[station],station))
    plt.tight_layout()
    fig.savefig(plot_output_path + '/seasonal_error/seasonal_error_station_%s' % str(station))
    plt.close()

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot error per station over years with savitzky_golay smoothing, if invalid errors are excluded
#-------------------------------------------------------------------------------------------------------------------------

def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
    The Savitzky-Golay filter removes high frequency noise from data.
    It has the advantage of preserving the original shape and
    features of the signal better than other types of filtering
    approaches, such as moving averages techniques.
    Parameters
    ----------
    y : array_like, shape (N,)
        the values of the time history of the signal.
    window_size : int
        the length of the window. Must be an odd integer number.
    order : int
        the order of the polynomial used in the filtering.
        Must be less then `window_size` - 1.
    deriv: int
        the order of the derivative to compute (default = 0 means only smoothing)
    Returns
    -------
    ys : ndarray, shape (N)
        the smoothed signal (or it's n-th derivative).
    Notes
    -----
    The Savitzky-Golay is a type of low-pass filter, particularly
    suited for smoothing noisy data. The main idea behind this
    approach is to make for each point a least-square fit with a
    polynomial of high order over a odd-sized window centered at
    the point.
    Examples
    --------
    t = np.linspace(-4, 4, 500)
    y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
    ysg = savitzky_golay(y, window_size=31, order=4)
    import matplotlib.pyplot as plt
    plt.plot(t, y, label='Noisy signal')
    plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
    plt.plot(t, ysg, 'r', label='Filtered signal')
    plt.legend()
    plt.show()
    References
    ----------
    .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
       Data by Simplified Least Squares Procedures. Analytical
       Chemistry, 1964, 36 (8), pp 1627-1639.
    .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
       W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
       Cambridge University Press ISBN-13: 9780521880688
    """
    from math import factorial
    
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except (ValueError, msg):
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

error_data = np.load(error_data_path)
n_stations, _, data_length = error_data.shape
year_length=365*8
x_times = get_time_axis(data_length,year_length)
station_name_dict = get_station_dict()
# loop over stations
for station in range(n_stations):
    data = [error_data[station,0,i:i + year_length] for i in range(0, data_length, year_length)]
    fig, axes = plt.subplots(len(data), figsize=(20, 20))
    min_y = np.min([e for e in error_data[station,0,:].flatten() if e < 1e10])
    max_y = np.max([e for e in error_data[station,0,:].flatten() if e < 1e10])
    for i in range(len(data)):
        # get start/end of the yearly periods
        start = x_times[i][0]
        end = x_times[i][0] + timedelta(days=365)
        # get the data and corresponding datetimes without the faulty samples
        x,error_filtered_data = get_filtered_data(x_times[i], data[i])
        # plot
        axes[i].plot(x, error_filtered_data, color='b', alpha=0.2)
        try:
        # get filtered line to show trend
            y = savitzky_golay(error_filtered_data, 87, 1, 0)
            axes[i].plot(x, y ,color='r')
        except ValueError:
            continue
        # set x-/y-limits
        axes[i].set_xlim((start,end))
        axes[i].set_ylim((min_y,max_y))

    axes[len(data)-1].set_xlabel('Days')
    plt.suptitle('%s (Station %s)' % (station_name_dict[station],station))
    plt.tight_layout()
    fig.savefig(plot_output_path + '/seasonal_error/savitzky_golay_seasonal_error_station_%s' % str(station))
    plt.close()

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot error per station over complete time series, if invalid errors are excluded
#-------------------------------------------------------------------------------------------------------------------------

error_data = np.load(error_data_path)
n_stations, _, data_length = error_data.shape
x_times = get_time_axis(data_length)
station_name_dict = get_station_dict()
# loop over stations
for station in range(n_stations):
    data = error_data[station,0,:]
    fig, axes = plt.subplots(1, figsize=(30, 10))
    min_y = np.min([e for e in error_data[station,0,:].flatten() if e < 1e10])
    max_y = np.max([e for e in error_data[station,0,:].flatten() if e < 1e10])
    # get start/end of the yearly periods
    start = x_times[0]
    end = x_times[-1]
    # get the data and corresponding datetimes without the faulty samples
    x,error_filtered_data = get_filtered_data(x_times, data)
    # plot
    axes.plot(x, error_filtered_data, color='b', alpha=0.2)
    try:
    # get filtered line to show trend
        b, a = signal.butter(3, 0.015)
        y = signal.filtfilt(b, a, error_filtered_data)
        axes.plot(x, y ,color='r')
    except ValueError:
        continue
    # set x-/y-limits
    axes.set_xlim((start,end))
    axes.set_ylim((min_y,max_y))

    axes.set_xlabel('Days')
    plt.suptitle('%s (Station %s)' % (station_name_dict[station],station))
    plt.tight_layout()
    fig.savefig(plot_output_path + '/seasonal_error/error_station_%s' % str(station))
    plt.close()

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot absolute error per station over years, if invalid errors are excluded
#-------------------------------------------------------------------------------------------------------------------------

error_data = np.abs(np.load(error_data_path))
n_stations, _, data_length = error_data.shape
year_length=365*8
x_times = get_time_axis(data_length,year_length)
station_name_dict = get_station_dict()
# loop over stations
for station in range(n_stations):
    data = [error_data[station,0,i:i + year_length] for i in range(0, data_length, year_length)]
    fig, axes = plt.subplots(len(data), figsize=(20, 20))
    max_y = np.max([e for e in error_data[station,0,:].flatten() if e < 1e10])
    for i in range(len(data)):
        # get start/end of the yearly periods
        start = x_times[i][0]
        end = x_times[i][0] + timedelta(days=365)
        # get the data and corresponding datetimes without the faulty samples
        x,error_filtered_data = get_filtered_data(x_times[i], data[i])
        error_filtered_data = error_filtered_data
        # plot
        axes[i].plot(x, error_filtered_data, color='b', alpha=0.2)
        try:
        # get filtered line to show trend
            b, a = signal.butter(3, 0.015)
            y = signal.filtfilt(b, a, error_filtered_data)
            axes[i].plot(x, y ,color='r')
        except ValueError:
            continue
        # set x-/y-limits
        axes[i].set_xlim((start,end))
        axes[i].set_ylim((0,max_y))

    axes[len(data)-1].set_xlabel('Days')
    plt.suptitle('%s (Station %s)' % (station_name_dict[station],station))
    fig.tight_layout(rect=[0, 0.03, 1, 0.97])
    fig.savefig(plot_output_path + '/seasonal_error/absolute_seasonal_error_station_%s' % str(station))
    plt.close()

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Plot absolute error per station over complete time series, if invalid errors are excluded
#-------------------------------------------------------------------------------------------------------------------------

error_data = np.abs(np.load(error_data_path))
n_stations, _, data_length = error_data.shape
x_times = get_time_axis(data_length)
station_name_dict = get_station_dict()
# loop over stations
for station in range(n_stations):
    data = error_data[station,0,:]
    fig, axes = plt.subplots(1, figsize=(30, 10))
    min_y = np.min([e for e in error_data[station,0,:].flatten() if e < 1e10])
    max_y = np.max([e for e in error_data[station,0,:].flatten() if e < 1e10])
    # get start/end of the yearly periods
    start = x_times[0]
    end = x_times[-1]
    # get the data and corresponding datetimes without the faulty samples
    x,error_filtered_data = get_filtered_data(x_times, data)
    # plot
    axes.plot(x, error_filtered_data, color='b', alpha=0.2)
    try:
    # get filtered line to show trend
        b, a = signal.butter(3, 0.015)
        y = signal.filtfilt(b, a, error_filtered_data)
        axes.plot(x, y ,color='r')
    except ValueError:
        continue
    # set x-/y-limits
    axes.set_xlim((start,end))
    axes.set_ylim((0,max_y))

    axes.set_xlabel('Days')
    plt.suptitle('%s (Station %s)' % (station_name_dict[station],station))
    plt.tight_layout()
    fig.savefig(plot_output_path + '/seasonal_error/absolute_error_station_%s' % str(station))
    plt.close()

## Cluster Stations by Error plottet on geographic Map

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Cluster stations by error
#-------------------------------------------------------------------------------------------------------------------------

def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

error_data = np.load(error_data_path).squeeze()
K=2
station_name_dict = get_station_dict()

idx_station_dict = {}

filtered_error_data = None
for i in range(error_data.shape[0]):
    if np.max(error_data[i]) > 1e10:
        continue
    if filtered_error_data is None:
        filtered_error_data = error_data[i]
        idx_station_dict[0] = i
    else:
        filtered_error_data = np.vstack((filtered_error_data, error_data[i]))
        idx_station_dict[len(filtered_error_data)-1] = i

OBS = nc.Dataset(observations)
station_x, station_y = OBS['lon'][:], OBS['lat'][:]

color_map = get_cmap(10)

# KMean
fig = plt.figure(figsize=(20,10))
clusteded_stations = KMeans(n_clusters=K).fit_predict(filtered_error_data)
colors = ([color_map(k*2) for _, k in enumerate(clusteded_stations)])
pylab.scatter(station_x,station_y, c=colors)
fig.savefig(plot_output_path + '/cluster_stations_by_error_k_means.png')
plt.close()

# Spectral
fig = plt.figure(figsize=(20,10))
clusteded_stations = SpectralClustering(n_clusters=K).fit_predict(filtered_error_data)
colors = ([color_map(k*2) for _, k in enumerate(clusteded_stations)])
pylab.scatter(station_x,station_y, c=colors)
fig.savefig(plot_output_path + '/cluster_stations_by_error_spectral.png')
plt.close()